In [1]:
import pyodbc as pyodbc
from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline
import yfinance as yf
from config import ROOT_DIR

#### Connect to sql server db and create stocks table

In [2]:
DRIVER = 'SQL Server'
SERVER_NAME = 'LAPTOP-S1EISQB0'
DB_NAME = 'newdb'

In [3]:
conn_str = (f"Driver={DRIVER};Server={SERVER_NAME};Database={DB_NAME};Trusted_Connection=yes;")
conn = pyodbc.connect(conn_str)
conn.autocommit = True 
cursor = conn.cursor()

In [4]:
query = """
use newdb;
drop table if exists stocks;
create table stocks (
Date date not null PRIMARY KEY,
Open_Price numeric(9, 6) not null,
High_Price numeric(9, 6) not null,
Low_Price numeric(9, 6) not null,
Close_Price numeric(9, 6) not null,
Adj_Close numeric(9, 6) not null,
Volume int not null
);
"""""

In [5]:
cursor.execute(query)

#### Get stocks data for Bank of America from yahoo finance api

In [6]:
yf.pdr_override()
startdate = datetime(2006,1,1)
enddate = datetime(2016,1,1)

In [7]:
data = pdr.get_data_yahoo(["BAC"], start=startdate, end=enddate)

[*********************100%***********************]  1 of 1 completed


In [8]:
data.reset_index(inplace=True)

In [9]:
data.to_csv(rf"{ROOT_DIR}\data\stocks.csv", index=False)

In [10]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-01-03,46.919998,47.180000,46.150002,47.080002,33.170311,16296700
1,2006-01-04,47.000000,47.240002,46.450001,46.580002,32.818039,17757900
2,2006-01-05,46.580002,46.830002,46.320000,46.639999,32.860313,14970700
3,2006-01-06,46.799999,46.910000,46.349998,46.570000,32.810982,12599800
4,2006-01-09,46.720001,46.970001,46.360001,46.599998,32.832123,15619400


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       2517 non-null   datetime64[ns]
 1   Open       2517 non-null   float64       
 2   High       2517 non-null   float64       
 3   Low        2517 non-null   float64       
 4   Close      2517 non-null   float64       
 5   Adj Close  2517 non-null   float64       
 6   Volume     2517 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 137.8 KB


#### Insert data to sql server table stocks

In [12]:
cols = ["Date", "Open_Price", "High_Price", "Low_Price", "Close_Price", "Adj_Close", "Volumne"]
records = data.values.tolist()

In [13]:
sql_insert = """ 
    insert into stocks
    values (?, ?, ?, ?, ?, ?, ?)
"""

In [14]:
cursor.executemany(sql_insert, records)

#### Rolling and expanding functions using pandas

In [15]:
data.set_index("Date", inplace=True)

In [16]:
open_rolling_agg = data[["Open"]].rolling(5, min_periods=3).agg(["mean", "sum", "max", "min"])

In [17]:
open_rolling_agg.head(10)

Open                                  
                 mean         sum        max        min
Date                                                   
2006-01-03        NaN         NaN        NaN        NaN
2006-01-04        NaN         NaN        NaN        NaN
2006-01-05  46.833333  140.500000  47.000000  46.580002
2006-01-06  46.825000  187.299999  47.000000  46.580002
2006-01-09  46.804000  234.020000  47.000000  46.580002
2006-01-10  46.700001  233.500004  47.000000  46.400002
2006-01-11  46.512001  232.560005  46.799999  46.060001
2006-01-12  46.440001  232.200005  46.799999  46.060001
2006-01-13  46.246001  231.230007  46.720001  45.830002
2006-01-17  45.982001  229.910007  46.400002  45.400002

In [18]:
data.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-01-03,46.919998,47.180000,46.150002,47.080002,33.170311,16296700
2006-01-04,47.000000,47.240002,46.450001,46.580002,32.818039,17757900
2006-01-05,46.580002,46.830002,46.320000,46.639999,32.860313,14970700
2006-01-06,46.799999,46.910000,46.349998,46.570000,32.810982,12599800
2006-01-09,46.720001,46.970001,46.360001,46.599998,32.832123,15619400
2006-01-10,46.400002,46.509998,45.880001,46.209999,32.557362,15634600
2006-01-11,46.060001,46.250000,45.750000,46.099998,32.479851,14742100
2006-01-12,46.220001,46.230000,45.709999,45.799999,32.268478,10546600
2006-01-13,45.830002,46.000000,45.680000,45.799999,32.268478,10791000


In [19]:
open_expanding_sum = data[["Open"]].expanding(5).sum()
open_expanding_mean = data[["Open"]].expanding(5).mean()
open_expanding_max = data[["Open"]].expanding(5).max()
open_expanding_min = data[["Open"]].expanding(5).min()

In [20]:
open_expanding_sum.head(10)

,Open
Date,
2006-01-03,NaN
2006-01-04,NaN
2006-01-05,NaN
2006-01-06,NaN
2006-01-09,234.020000
2006-01-10,280.420002
2006-01-11,326.480003
2006-01-12,372.700005
2006-01-13,418.530006


In [21]:
data.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2006-01-03,46.919998,47.180000,46.150002,47.080002,33.170311,16296700
2006-01-04,47.000000,47.240002,46.450001,46.580002,32.818039,17757900
2006-01-05,46.580002,46.830002,46.320000,46.639999,32.860313,14970700
2006-01-06,46.799999,46.910000,46.349998,46.570000,32.810982,12599800
2006-01-09,46.720001,46.970001,46.360001,46.599998,32.832123,15619400
2006-01-10,46.400002,46.509998,45.880001,46.209999,32.557362,15634600
2006-01-11,46.060001,46.250000,45.750000,46.099998,32.479851,14742100
2006-01-12,46.220001,46.230000,45.709999,45.799999,32.268478,10546600
2006-01-13,45.830002,46.000000,45.680000,45.799999,32.268478,10791000


#### Rolling and expanding aggregates using sql server

In [22]:
rolling_sum_query = """ 
    select Date,
    sum(Open_Price) over (order by Date rows between 4 preceding and current row) Rolling_Open_Sum_Rows,
    Open_Price
    from stocks
"""

In [24]:
rolling_sum = pd.read_sql(rolling_sum_query, conn)

c:\Users\meeta\OneDrive\Documents\pyscripts\.myenv\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [25]:
rolling_sum

,Date,Rolling_Open_Sum_Rows,Open_Price
0,2006-01-03,46.919998,46.919998
1,2006-01-04,93.919998,47.000000
2,2006-01-05,140.500000,46.580002
3,2006-01-06,187.299999,46.799999
4,2006-01-09,234.020000,46.720001
...,...,...,...
2512,2015-12-24,85.700000,17.320000
2513,2015-12-28,85.729998,17.219999
2514,2015-12-29,85.999998,17.250000
2515,2015-12-30,86.150000,17.200001
